# Importation

In [97]:
from tqdm import tqdm
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
import nltk
from nltk import pos_tag, RegexpParser, word_tokenize
# Download NLTK resources
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import time
import warnings
warnings.filterwarnings("ignore")
import sys
import re
import pandas as pd
from PyPDF2 import PdfReader
import cv2
import numpy as np
from PIL import ImageGrab
import time
import os
import pytesseract
from PIL import Image
import fitz
import cv2
import pytesseract
from PIL import Image
import nltk
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hamma\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
from pdf2image import convert_from_path

In [69]:
pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 131.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 130.4 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 217.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to buil

In [70]:
import pyautogui

# Transforming to pictures

In [3]:
# PDF file paths
pdfPMBOK = "PMBOK6-2017.pdf" #431-494
pdfPMI = "practice-standard-project-risk-management.pdf" #1-116
output_folder_1="PMBOOKimgs"
output_folder_2="Practiceimgs"

In [42]:
def convert_pdf_to_images(pdf_file_path, output_folder, start_page, end_page):
    images = convert_from_path(pdf_file_path, output_folder=output_folder, fmt='png', first_page=start_page, last_page=end_page, output_file="page")

    print("Done.")

In [43]:
convert_pdf_to_images(pdfPMBOK, output_folder_1, 431, 494)

Done.


In [44]:
convert_pdf_to_images(pdfPMI, output_folder_2, 1, 116)

Done.


# Extracting Figures

### YOLO: (does not work)

###### must install yolov3.weights yolov3.cfg coco.names (ask chatgpt)

In [80]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load class names
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Load image
img = cv2.imread("PMBOOKimgs/page0001-446.png")
height, width, channels = img.shape

# Perform object detection
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(net.getUnconnectedOutLayersNames())

# Information to store detected objects
class_ids = []
confidences = []
boxes = []

# Minimum confidence threshold
conf_threshold = 0.5

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Non-maximum suppression to remove overlapping boxes
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Capture screenshots of detected objects
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]

        # Draw bounding box and label on the image
        color = (0, 255, 0)
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Crop the object and save it as a screenshot
        object_roi = img[y:y + h, x:x + w]
        cv2.imwrite(f"{label}.png", object_roi)

# Scale factor for display
scale_factor = 0.3  # Adjust this value as needed

# Resize the image
resized_img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor)

# Display the resized image with bounding boxes
cv2.imshow("Image", resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [103]:
yolo_object_detection("PMBOOKimgs/page0001-432.png")

NameError: name 'yolo_object_detection' is not defined

### openCV:

In [26]:
def detect_shapes(image_path, min_contour_area=1000):
    # Read the image
    image = cv2.imread("PMBOOKimgs/"+image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    k=0
    for contour in contours:
        k=k+1
        # Calculate the contour area
        area = cv2.contourArea(contour)

        # If the contour area is greater than the minimum threshold
        if area >= min_contour_area:
            # Approximate the contour as a polygon
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the polygon has 4 vertices, indicating a rectangle
            if len(approx) == 4:
                # Get the bounding box coordinates of the rectangle
                x, y, w, h = cv2.boundingRect(contour)

                # Define a region of interest (ROI) using the rectangle coordinates
                roi = image[y:y+h, x:x+w]

                # Save the ROI as an image for inspection
                cv2.imwrite("screenshots/"+image_path+"/"+str(k)+"detected__"+image_path, roi)                
                

In [123]:
detect_shapes("page0001-445.png")

In [124]:
for k in os.listdir("PMBOOKimgs/"):
    os.makedirs("screenshots/"+k, exist_ok=True)
    detect_shapes(k)

# Extracting Texts from images

In [4]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\hamma\AppData\Local\Tesseract-OCR\tesseract.exe'

In [30]:
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

In [41]:
def extract_text(image_path):
    # Read the image
    image = cv2.imread(image_path)
    

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use adaptive thresholding to segment text from the background
    _, threshold = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Remove horizontal and vertical lines using morphological operations
    kernel = np.ones((4, 4), np.uint8)
    clean_image = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel)
    text = pytesseract.image_to_string(Image.fromarray(threshold))


    print(text)
    # Save the extracted text to a file
    #with open(output_text_path, 'w', encoding='utf-8') as text_file:
        #text_file.write(text)

In [44]:
extract_text("screenshots/page0001-446.png/54detected__page0001-446.png")

Project
Management
Plan

Project management plan

« Requirements management plan
« Schedule management plan
« Cost management plan

« Resource management plan
* Quality management plan
« Risk management plan
« Scope baseline

* Schedule baseline

* Cost baseline

Project
Documents

reject documents
Assumption log
Cost estimates 11.2

Issue log Risks
Lessons learned register

Project
eceeeseserseseeseneces: Docurnents

Risk register
* Risk report

Requirements documentation
Resource requirements.
Stakeholder register

Pr
.
.
* Duration estimates Identify
.
.
.
.
.

12.1
Plan
Procurement
Management

Procurement documentation

12.2
Conduct
Procurements

Agreements

Enterprise/
Organization

Enterprise environmental factors.
Organizational process assets

Project documents updates
« Assumption log

* Issue log

« Lessons learned register




# Extracting Texts from pdf

In [ ]:


def extract_text_and_figures(pdf_path, output_text_path, output_figures_path):
    doc = fitz.open(pdf_path)
    text_output = []

    for page_number in range(doc.page_count):
        page = doc[page_number]
        images = page.get_images(full=True)

        for img_index, image in enumerate(images):
            base_image = doc.extract_image(image)
            image_bytes = base_image["image"]
            image_path = f"figure_page{page_number + 1}_img{img_index + 1}.png"

            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)

            # OCR on the image
            img_cv2 = cv2.imread(image_path)
            gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
            _, threshold = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
            text = pytesseract.image_to_string(Image.fromarray(threshold))

            # Save the extracted text
            text_output.append(f"Page {page_number + 1}, Figure {img_index + 1}:\n{text}\n")

    # Save the extracted text to a file
    with open(output_text_path, 'w', encoding='utf-8') as text_file:
        text_file.writelines(text_output)

    doc.close()


In [9]:
import fitz
import pandas as pd

def extract_plain_text_from_pdf(pdf_file_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_file_path)

    # Initialize an empty list to store the extracted data
    data = []

    for page_number in range(431, 495):
        page = pdf_document.load_page(page_number)
        text = page.get_text()
        blocks = text.split("\n")

        # Initialize variables to store the current title and description
        current_title = ""
        current_description = ""

        for block in blocks:
            block = block.strip()

            if "Figure" in block:
                continue
            elif block.isupper():
                # If a new title is found, add the previous title and description to the data list
                if current_title:
                    data.append({"title": current_title, "description": current_description})
                current_title = block
                current_description = ""
            elif block.istitle():
                # If a new title is found, add the previous title and description to the data list
                if current_title:
                    data.append({"title": current_title, "description": current_description})
                current_title = block
                current_description = ""
            elif block.startswith("- "):
                continue
            else:
                # Append the block to the current description
                current_description += block + "\n"

        # Add the last title and description to the data list
        if current_title:
            data.append({"title": current_title, "description": current_description})

    # Close the PDF document
    pdf_document.close()

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data)

    return df


In [10]:
# Extract plain text and create DataFrame
df = extract_plain_text_from_pdf(pdfPMBOK)

In [11]:
df

,title,description
0,Part 1 - Guide,
1,.1 Inputs,.1 Project charter\n.2 Project management plan...
2,.2 Tools & Techniques,.1 Expert judgment\n.2 Data analysis\n
3,.3 Meetings,
4,.3 Outputs,.1 Risk management plan\n
...,...,...
476,11.7.3.3 PROJECT MANAGEMENT PLAN UPDATES,Any change to the project management plan goes...
477,Part 1 - Guide,
478,11.7.3.4 PROJECT DOCUMENTS UPDATES,Project documents that may be updated as a res...
479,11.7.3.5 ORGANIZATIONAL PROCESS ASSETS UPDATES,Organizational process assets that are updated...


In [14]:
df.to_excel("data.xlsx", index=False)

# Cleaning Data and Normalization:

In [46]:
def clean_text(text):
    # Remove special characters and symbols (retain letters, numbers, and basic punctuation)
    cleaned_text = re.sub(r'[^a-zA-Z0-9.,!? ]', '', text)
    
    # Remove extra spaces and trim leading/trailing spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text.lower()

#  Tokenization :

In [81]:
from nltk.tokenize import word_tokenize
def tokenize(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    return tokens

#  POS-TAGGING

In [82]:
def pos_tagging(text):
    # Tokenize the text into words
    words = tokenize(text)

    # Perform POS tagging
    pos_tags = pos_tag(words)

    return pos_tags

#  STOPWORDS

In [304]:
def remove_stopwords_from_list(text):
    
    pos_tags=pos_tagging(text)
    word_list=tokenize(text)
    # Define a list of POS tags for words to keep (e.g., nouns and adjectives)
    allowed_pos_tags = ['VBD',  # Verb, past tense
        'VBG',  # Verb, gerund or present participle
        'VBN',  # Verb, past participle
        'VBP',  # Verb, non-3rd person singular present
        'VBZ',  # Verb, 3rd person singular present
        'PRP','NN','VB',"MD", 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS',",",".","!","?"]  # Nouns and adjectives
    
    # Filter out stopwords based on POS tags
    filtered_words = [word for word, pos_tag in zip(word_list, pos_tags) if pos_tag[1]
                      in allowed_pos_tags]
    
    return ' '.join(filtered_words)

#  Chunking :

In [372]:
import networkx as nx
import matplotlib.pyplot as plt
def chunk_words_by_meaning(text):
    # Tokenize the text into words
    words = tokenize(text)

    # Perform POS tagging
    pos_tags = pos_tagging(text)
    chunks = []
    # Define a grammar for chunking (NP: Noun Phrase)
    # Define a grammar for chunking
    grammar = r"""
        NP: {<JJ.*>*<PRP|NN.*>+}   # chunk determiners, adjectives, personal pronouns, and nouns
        VP: {<MD>*<VB.*>+}      # chunk verb phrases with optional NP
        CHUNK: {<NP><VP>+<NP>}  # chunk relation with NP-VP-NP pattern
    """
    
    # Create a chunk parser with the defined grammar
    
    chunk_parser = RegexpParser(grammar)

    
    # Apply chunking to POS-tagged words
    tree = chunk_parser.parse(pos_tags)

        # Extract phrases from the tree
    for subtree in tree.subtrees():
        if subtree.label() == 'CHUNK':
            np_vp = [token for token, _ in subtree.leaves() if 'NP' in token or 'VP' in token]
            print(type(subtree))
            print(subtree)
            chunks.append(subtree)

    return chunks,tree


#### after this phase it should be returned a list of chunks cleaned

In [374]:
text= df["description"].loc[478]

In [375]:
text=clean_text(text)
text

'project documents that may be updated as a result of carrying out this process include but are not limited touu assumption log. described in section 4.1.3.2. during the monitor risks process, new assumptions may bemade, new constraints may be identied, and existing assumptions or constraints may be revisited and changed.the assumption log is updated with this new information.uu issue log. described in section 4.3.3.3. where issues are identied as part of the monitor risks process, theseare recorded in the issue log.uu lessons learned register. described in section 4.4.3.1. the lessons learned register is updated with anyriskrelated lessons learned during risk reviews so these can be used on later phases of the project or infuture projects.uu risk register. described in section 11.2.3.1. the risk register is updated with information on individual projectrisks generated during the monitor risks process. this may include adding new risks, updating outdated risks orrisks that were realize

In [376]:
text=remove_stopwords_from_list(text)

In [377]:
text

'project documents may be updated result carrying process include are limited touu assumption log . described section monitor risks process , new assumptions may bemade , new constraints may be identied , existing assumptions constraints may be revisited changed.the assumption log is updated new information.uu issue log . described section issues are identied part monitor risks process , theseare recorded issue log.uu lessons learned register . described section lessons learned register is updated anyriskrelated lessons learned risk reviews can be used later phases project infuture projects.uu risk register . described section risk register is updated information individual projectrisks generated monitor risks process . may include adding new risks , updating outdated risks orrisks were realized , updating risk responses , forth.uu risk report . described section new information becomes available monitor risksprocess , risk report is updated reect current status major individual projec

In [378]:
text,chunks=chunk_words_by_meaning(text)

<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP project/NN documents/NNS)
  (VP may/MD be/VB updated/VBN)
  (NP result/NN))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP process/NN include/NN)
  (VP are/VBP)
  (NP limited/JJ touu/JJ assumption/NN log/NN))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP assumptions/NNS constraints/NNS)
  (VP may/MD be/VB revisited/VBN)
  (NP changed.the/JJ assumption/NN log/NN))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP described/JJ section/NN issues/NNS)
  (VP are/VBP)
  (NP identied/JJ part/NN monitor/NN risks/NNS process/NN))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP theseare/NN)
  (VP recorded/VBN)
  (NP issue/NN log.uu/NN lessons/NNS))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP described/JJ section/NN lessons/NNS)
  (VP learned/VBD)
  (NP register/NN))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP anyriskrelated/JJ lessons/NNS)
  (VP learned/VBD)
  (NP risk/NN reviews/NNS))
<class 'nltk.tree.tree.Tree'>
(CHUNK
  (NP phases/NNS)
  (VP project/VBP)
  (NP infuture/

In [353]:
text

[Tree('CHUNK', [Tree('NP', [('project', 'NN'), ('documents', 'NNS')]), Tree('VP', [('may', 'MD'), ('be', 'VB'), ('updated', 'VBN')]), Tree('NP', [('result', 'NN')])]),
 Tree('CHUNK', [Tree('NP', [('process', 'NN'), ('include', 'NN')]), Tree('VP', [('are', 'VBP')]), Tree('NP', [('limited', 'JJ'), ('touu', 'JJ'), ('assumption', 'NN'), ('log', 'NN')])]),
 Tree('CHUNK', [Tree('NP', [('assumptions', 'NNS'), ('constraints', 'NNS')]), Tree('VP', [('may', 'MD'), ('be', 'VB'), ('revisited', 'VBN')]), Tree('NP', [('changed.the', 'JJ'), ('assumption', 'NN'), ('log', 'NN')])]),
 Tree('CHUNK', [Tree('NP', [('described', 'JJ'), ('section', 'NN'), ('issues', 'NNS')]), Tree('VP', [('are', 'VBP')]), Tree('NP', [('identied', 'JJ'), ('part', 'NN'), ('monitor', 'NN'), ('risks', 'NNS'), ('process', 'NN')])]),
 Tree('CHUNK', [Tree('NP', [('theseare', 'NN')]), Tree('VP', [('recorded', 'VBN')]), Tree('NP', [('issue', 'NN'), ('log.uu', 'NN'), ('lessons', 'NNS')])]),
 Tree('CHUNK', [Tree('NP', [('described', 'J

#  Relationship & Concepts :

In [312]:
from collections import Counter  # Import the Counter class
def extrac_relations(chunked_text):
        # Extract and print identified concepts and relationships
    concepts = []
    relationships = []

    for subtree in chunked_text.subtrees():
        if subtree.label() == 'NP':
            concepts.append(" ".join(word for word, pos in subtree.leaves()))
        elif subtree.label() == 'VP':
            relationships.append(" ".join(word for word, pos in subtree.leaves()))

    # Concept frequencies
    concept_freq = Counter(concepts)
    frequency_threshold = 2  # Concepts that occur at least 10 times
    pertinent_concepts = [concept for concept in concepts if concept_freq[concept] >= frequency_threshold]

    print(f"Total Concepts: {len(concepts)}")
    print(f"Total Pertinent Concepts: {len(pertinent_concepts)}")
    print(f"Total Relationships: {len(relationships)}")
    print("================================================================")
    print("Concepts:", concepts)
    print("Relationships:", relationships)
    print("Pertinent Concepts :", pertinent_concepts)
    return pertinent_concepts

In [325]:
for index, row in df.iterrows():
    text = row["description"]
    cleaned_text = remove_stopwords_from_list(clean_text(text))
    chunks,tree = chunk_words_by_meaning(cleaned_text)
    extrac_relations(tree)
    

Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 2
Total Pertinent Concepts: 0
Total Relationships: 1
Concepts: ['.1 project charter.2 project management plan.3 project', 'environmentalfactors.5 organizational processassets']
Relationships: ['documents.4']
Pertinent Concepts : []
Total Concepts: 1
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: ['.1 expert judgment.2 data analysis']
Relationships: []
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 1
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: ['.1 risk management plan']
Relationships: []
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Co

Total Concepts: 41
Total Pertinent Concepts: 0
Total Relationships: 30
Concepts: ['focus project risk management', 'types risk', 'projectrisks', 'understood wider context', 'trends', 'practices project risk management', 'arenot', 'limited touu nonevent risks', 'most projects', 'risks', 'uncertain future events', 'eventbased risks', 'key seller', 'business project', 'customer maychange requirement design', 'subcontractor', 'enhancements', 'processes.there', 'recognition nonevent risks', 'maintypes nonevent risksun variability risk', 'uncertainty', 'key characteristics', 'event activity', 'variability risks', 'productivity', 'target', 'number errors', 'testing', 'unseasonal weather conditions', 'theconstruction phase.un ambiguity risk', 'uncertainty exists', 'future', 'areas', 'whereimperfect knowledge', 'projects ability', 'objectives', 'elements', 'therequirement technical solution', 'future developments', 'regulatory frameworks', 'inherent systemiccomplexity project']
Relationships: [

Total Concepts: 46
Total Pertinent Concepts: 3
Total Relationships: 29
Concepts: ['risk management plan', 'component project management plan', 'risk managementactivities', 'risk management plan', 'elementsuu risk strategy', 'general approach', 'risk project.uu methodology', 'denes', 'specic approaches', 'tools', 'data sources', 'perform riskmanagement project.uu', 'responsibilities', 'denes lead', 'support', 'risk management team members', 'ofactivity', 'risk management plan', 'claries', 'funding', 'identies funds', 'perform activities', 'project risk management', 'application contingency management reserves.uu timing', 'denes project risk management processes', 'theproject life cycle', 'risk management activities inclusion project schedule.uu risk categories', 'individual project risks', 'common way structure risk categoriesis risk', 'structure rbs', 'hierarchical representation potential sources', 'seeproject risks', 'risks', 'identied risks', 'theorganization', 'generic rbs', 'proje

Total Concepts: 11
Total Pertinent Concepts: 3
Total Relationships: 9
Concepts: ['project management plan requirements management plan', 'management plan', 'management plan', 'management plan quality management plan', 'management plan', 'baseline schedule baseline cost', 'baselineproject documents assumption log cost estimates', 'estimates', 'log lessons', 'register requirements documentation resource requirements', 'register procurement documentation12.2']
Relationships: ['schedule', 'cost', 'resource', 'risk', 'scope', 'duration', 'issue', 'learned', 'stakeholder']
Pertinent Concepts : ['management plan', 'management plan', 'management plan']
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []

Total Concepts: 63
Total Pertinent Concepts: 8
Total Relationships: 39
Concepts: ['techniques', 'process include', 'goal', 'see section', 'comprehensive list', 'individualproject risks sources', 'overall project risk', 'project team performs', 'witha multidisciplinary set experts', 'part team', 'ideas', 'guidance afacilitator', 'brainstorm session', 'structured techniques', 'categories', 'such risk breakdown structure', 'framework', 'particular attention', 'risks', 'technique', 'formed.uu checklists', 'checklist', 'list items', 'actions', 'points', 'it', 'reminder.risk checklists', 'historical information knowledge', 'fromsimilar projects', 'other sources information', 'they', 'effective way capture lessons', 'projects', 'specic individual project risks', 'thatmay', 'relevant project', 'organization', 'risk checklist', 'own completedprojects', 'generic risk checklists industry', 'checklist', 'quick simple use', 'it', 'impossible build exhaustive', 'care', 'checklist', 'avoidthe effort 

...

Total Concepts: 60
Total Pertinent Concepts: 6
Total Relationships: 32
Concepts: ['project documents', 'outputs process include', 'risk', 'section risk report', 'reect results', 'quantitative risk analysis', 'includeuu assessment', 'overall project risk exposure', 'overall project risk', 'key measuresun chances project success', 'indicated probability project', 'key objectives', 'required end date', 'interim milestones', 'cost target', 'identied individual project', 'other sources uncertainty', 'degree inherent variability', 'project time analysis', 'indicatedby range', 'possible project outcomes.uu', 'probabilistic analysis project', 'key outputs', 'quantitative risk analysis', 'such scurves', 'tornado diagrams', 'criticality analysis', 'narrative interpretation results.possible', 'detailed results', 'quantitative risk analysis', 'amount contingency reserve', 'level condenceun identication', 'individual project', 'other sources uncertainty', 'greatest effect theproject', 'critical pat

Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 12
Total Pertinent Concepts: 0
Total Relationships: 6
Concepts: ['section expertise', 'individuals groups', 'knowledgein', 'topicsuu threat response strategies', 'uu opportunity response strategies', 'uu contingent response strategies', 'overall project risk response strategies.expert input', 'individuals', 'particular subject matter expertise', 'relevant specic individualproject risk', 'example', 'specialist technical knowledge']
Relationships: ['described', 'should be considered', 'specialized', 'following', 'may be sought', 'is required']
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 12
Total Pertinent Concepts: 0
Total Relationships: 8
Concepts: ['techniques', 'process include', 'limited interviews seesection development respons

Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 56
Total Pertinent Concepts: 5
Total Relationships: 49
Concepts: ['project documents', 'result', 'process include', 'limited touu assumption log', 'section plan', 'responses process', 'new assumptionsmay', 'new constraints', 'assumptions constraints', 'assumption log', 'new information.uu cost forecasts', 'section cost forecasts', 'result', 'risk responses.uu lessons', 'register', 'described section lessons', 'register', 'risk responses', 'useful future phases', 'future projects.uu project schedule', 'described section activities', 'agreedupon risk responses', 'project schedule.uu project team assignments', 'section responses', 'necessaryresources', 'action', 'risk response plan', 'resources', 'experienced personnel', 'agreedupon action project team specicbudget time allowance action', 'technical resources', 'complete action.uu risk register', 'sec

Total Concepts: 6
Total Pertinent Concepts: 0
Total Relationships: 4
Concepts: ['described section project management information systems', 'schedule', 'resource', 'risk response plans', 'activities', 'other project activities']
Relationships: ['can include', 'ensure', 'associated', 'are integrated']
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 7
Total Pertinent Concepts: 0
Total Relationships: 6
Concepts: ['section implementation risk responses', 'request cost andschedule', 'other components', 'management plan', 'change requests', 'reviewand disposition perform', 'change control process section']
Relationships: ['described', 'may result change', 'baselines', 'project', 'are processed', 'integrated']
Pertinent Concepts : []
Total Concepts: 0
Total Pertinent Concepts: 0
Total Relationships: 0
Concepts: []
Relationships: []
Pertinent Concepts : []
Total Concepts: 37
Tota

KeyboardInterrupt: 

In [326]:
chunks

[([Tree('NP', [('described', 'JJ'), ('section', 'NN'), ('project', 'NN'), ('management', 'NN'), ('plan', 'NN'), ('components', 'NNS')]),
   Tree('VP', [('include', 'VBP'), ('are', 'VBP')]),
   Tree('NP', [('limited', 'JJ'), ('riskmanagement', 'NN'), ('plan', 'NN')])],
  [Tree('NP', [('limited', 'JJ'), ('riskmanagement', 'NN'), ('plan', 'NN')]),
   Tree('NP', [('described', 'JJ'), ('section', 'NN'), ('project', 'NN'), ('management', 'NN'), ('plan', 'NN'), ('components', 'NNS')])]),
 ([Tree('NP', [('described', 'JJ'), ('section', 'NN'), ('risk', 'NN'), ('management', 'NN'), ('plan', 'NN')]),
   Tree('VP', [('provides', 'VBZ')]),
   Tree('NP', [('guidance', 'NN'), ('whenrisks', 'NNS')])],
  [Tree('NP', [('guidance', 'NN'), ('whenrisks', 'NNS')]),
   Tree('NP', [('described', 'JJ'), ('section', 'NN'), ('risk', 'NN'), ('management', 'NN'), ('plan', 'NN')])]),
 ([Tree('NP', [('responsibilities', 'NNS')]),
   Tree('VP', [('themonitoring', 'VBG')]),
   Tree('NP', [('process', 'NN')])],
  [Tree

#  Frequencies :